# 4. მონაცემების დამუშავება მოდელირებისთვის

წინა თავში ჩვენ გავიარეთ მონაცემთა კვლევისა და ვიზუალიზაციის ტექნიკები. ვნახეთ თუ რამდენად სიღრმისეულად შეიძლება გავაანალიზოთ ერთი ცვლადიც კი. თუმცა ბევრი მნიშნველოვანი ნაბიჯი გამოვტოვეთ.

რეალურ ცხოვრებაში მონაცემების კვლევისა და მოდელირებისთვის მონაცემების სწორად მომზადების პროცესი ერთმანეთზე გადაჭაჭვულია, მეტიც შეგვიძლია ვთქვათ რომ ერთი მთლიანი პროცესია. თუმცა იმისათვის რომ ყველა საჭირო ტექნიკა დავფაროთ და ხაზი გავუსვათ ორივე საკითხის მნიშნველოვნებას, გადავწყვიტეთ ისინი გაგვეყო და ორ თავად წარმოგვედგინა. 

შესაბამისად მოცემული თავი შესაძლოა არ იყოს ისეთივე "ჩამთრევი" როგორც წინა. შესაძლოა ცოტა "მშრალადაც" მოგეჩვენოთ, თუმცა მისი წილი გამართული მოდელის ასაგებად უაღრესად მნიშნველოვანია. მომდევნო აბზაცებში ჩვენ შევეხებით ისეთ საკითხებს როგორებიცაა: ზედმეტი ცვლადების მოშორება, მონაცემებში სიცარიელეების შევსება, განცალკვებულ ჩანაწერებთან გამკლავება, ახალი ცვლადების გამოყვანა არსებული ინფორმაციიდან, კატეგორიული ცვლადების გარდაქმნა მოდელისთვის მისაღებ ფორმაში და მთლიანი მატრიცის სკალირება.

*შენიშვნა: kaggle.com-ზე მოცემული მონაცემები ხშირ შემთხვევაში უკვე "გაწმედილია", ჩვენ კი ყველა ზემოთ ხსენებული ტექნიკის უკეთ შესასწავლად და რეალურ გარემოსთან მეტად მისაახლოვებლად მონაცემებს განზრახ დავაზიანებთ. აღნიშნული პროცესი ცდება წიგნის ფარგლებს, ამიტომ ჩვენ პირდაპირ "დაზიანებულ" მონაცემებს მოგაწვდით* 

### 4.1 მონაცემების გაწმენდა

პირველ რიგში დავიწყოთ გაწმენდით და მოვიშოროთ ყველა არა საჭირო ცვლადი. ამ ყველაფრისთვის ჩვენთვის უკვე კარგად ნაცნოიბ ბიბლიოთეკებს გამოვიყენებთ:

In [133]:
import pandas as pd #მონაცემების დამუშავების ბიბლიოთეკა
import numpy as np #მონაცემების დამუშავების ბიბლიოთეკა
import seaborn as sns #მონაცემების ვიზუალიზაციის ბიბლიოთეკა
import matplotlib.pyplot as plt #მონაცემების ვიზუალიზაციის ბიბლიოთეკა
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
%matplotlib inline

In [39]:
df = pd.read_csv('NYCTaxi_damaged.csv') #მთლიანი მონაცემების შემოტანა სამუშაო გარემოში

In [40]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2.0,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.982155,40.767937,NaN,40.765602,NaN,455
1,id2377394,1.0,2016-06-12 00:43:35,2016-06-12 00:54:38,1.0,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2.0,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2.0,NaN,2016-04-06 19:39:40,1.0,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,N/a,2016-03-26 13:30:55,2016-03-26 13:38:10,NaN,-73.973053,40.793209,-73.972923,40.782520,N,435


როგორც ვხედავთ მოცემული გვაქვს ჩანაწერის აიდის ცვლადი, რომელიც უნიკალურია ყველა ჩანაწერისთვის. მსგავსი ტიპის ცვლადები არანაირ ღირებულებას არ ქმნიან, შესაბამისად პირველ რიგში მისი წაშლით დავიწყოთ:

In [41]:
df.drop(['id'], axis = 1, inplace = True)
# .drop() ფუნქცია შლის მონაცემებს, axis არგუმენტით ვაზუსტებთ სვეტის წაშლა გვინდა თუ სტრიქონის. 1 შეესაბამება
# სვეტს სწორად ამიტომ გავუტოლეთ axis 1-ს. ხოლო  inplace პარამეტრის (რომელიც პანდას ბევრ ფუნქციას მოყვება)
# True-ზე გატოლებით ჩვენ ვეუბნებით რომ მითითებული ბრძანებით df-ს თავზე გადააწეროს ახალი მნიშნვლეობა. 

შემდეგი ნაბიჯია ისეთი ველების წაშლა, რომელთა დიდი ნაწილი არის შეუვსებელი. ჩვენ შემთხვევაში ასე მოვიქცევით 70%-ზე მეტი ცარიელი ველების მქონდე ცვლადებზე. თუმცა სანამ ველებს შევამოწმებთ შევსების პროცენტულობაზე, მნიშნველოვანია დარწმუნებულები ვიყოთ, რომ ცარიელი მონაცემები შესაბამის ფორმატშია ჩაწერილი. ხშირად სხვადასხვა სისტემები სხვადასხვაგვარ მნიშნველობებს ანიჭებენ სიცარიელებს: Nan, NaN, N/A, None და ა.შ. ხოლო პანდას ფუნქცია რომელიც აღნიშნულს ამოწმებს მხოლოდ გაკრვეულ ტიპებს აღიქვამს. ამიტომ პირველ რიგში დავწეროთ ფუნქცია რომელიც შეამოწმებს ჩამოთვლილთაგან რომელიმე ხო არ ხვდება მონაცემებში და თუ ხვდება მათ ჩაანაცვლებს np.nan მნიშნველობებით.

*შენიშვნა: ზოგ შემთხვევაში ცარიელი მნიშვნელობა შესაძლოა შინაარსობრივად 0-ის ტოლი იყოს. მაგ. თუ მომხმარებლების ტრანზაციების ინფორმაცია გვაქვს და მნიშვნელობები ცარიელია, ეს იმას ნიშნავს რომ მათ არ განუხორციელებიათ ტრანზაქცია შესაბამისად ასეთი ველები 0-ით უნდა შევავსოთ და არც ერთ შემთხვევაში ასეთი ცვლადი არ უნდა წავშალოთ მცირედით შევსებულობის მიზეზით*

სანამ ფუნქციას დავწერთ მოდით მანამდე ერთ მარტივ ხრიკს მივმართოთ რომელიც დაგვეხმარება ზემოთ აღნიშნული "არა სწორი" ფორმატით ჩაწერილი ცარიელი მნიშვნელობების დაიდენტიფიცირებაში, დავბეჭდოთ სვეტების ტიპები .info() ფუნქციის გამოყენებით.

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   vendor_id           1458644 non-null  object 
 1   pickup_datetime     1312567 non-null  object 
 2   dropoff_datetime    1458644 non-null  object 
 3   passenger_count     1312998 non-null  float64
 4   pickup_longitude    1458644 non-null  float64
 5   pickup_latitude     1458644 non-null  float64
 6   dropoff_longitude   1312398 non-null  float64
 7   dropoff_latitude    1312513 non-null  float64
 8   store_and_fwd_flag  1312672 non-null  object 
 9   trip_duration       1458644 non-null  int64  
dtypes: float64(5), int64(1), object(4)
memory usage: 111.3+ MB


როგორც ვხედავთ ყველა ცვლადი "ლოგიკურ" ფორმატშია გარდა vendor_id-სა. როდესაც ცხრილის პირველი 5 ჩანაწერი დავბეჭდეთ ნათლად გამოჩნდა, რომ აღნიშნულ ცვლადშიც რიცხვითი მნიშნველობები იყო ჩაწერილი. ეს კი შეიძლება მინიშნება იყოს ზუსტადაც პანდასთვის "მიუღებელ" ფორმატში ჩაწერილი ცარიელი მნიშვნელობების. მოდით ჯერ შევამოწმოთ ცარიელი მნიშვნელობები, შევამოწმოთ გვაქვს თუ არა vendor_id-ის ცვლადში, შემდგომ დავწეროთ ფუნქცია რომელიც ყველა არასასრუველ ფორმატს გარდაქმნის ჩვენთვის სასურველში, გადავატაროთ მონაცმებს და შევამოწმოთ შეგვეცვლება თუ არა რაიმე.

In [43]:
df.isnull().mean() * 100

# როგორც ვხედავთ გარკვეული მონაცემები 10%-ის ფარგლებში არის ცარიელი, თუმცა "ეჭვმიტანილი" vendor_id ცვლადი
# ერთი შეხედვით სრულად შევსებულია

vendor_id              0.000000
pickup_datetime       10.014575
dropoff_datetime       0.000000
passenger_count        9.985027
pickup_longitude       0.000000
pickup_latitude        0.000000
dropoff_longitude     10.026161
dropoff_latitude      10.018277
store_and_fwd_flag    10.007377
trip_duration          0.000000
dtype: float64

In [126]:
def replace_most_pop_miss(X):
    dict_nans = dict.fromkeys(['?', 'n/a', 'nan', 'NaN', ',', 'N/a', 'N/A', 'none', '*', '.'], np.nan)
    objects_list = X.select_dtypes(include = ['object']).columns
    X[objects_list] = X[objects_list].replace(dict_nans)
    return X

In [127]:
df = replace_most_pop_miss(df)

In [128]:
df.isnull().sum() / df.shape[0] * 100

vendor_id              9.960004
pickup_datetime       10.014575
dropoff_datetime       0.000000
passenger_count        9.985027
pickup_longitude       0.000000
pickup_latitude        0.000000
dropoff_longitude     10.026161
dropoff_latitude      10.018277
store_and_fwd_flag    10.007377
trip_duration          0.000000
dtype: float64

ზემოთ მოყვანილი ცხრილიდან ჩანს, რომ არც ერთი ცვლადი არ იმსახურებს წაშლას (70% პრინციპის გამოყენებით) თუმცა ჩვენ მაინც დავწეროთ ფუნქცია რომელიც წაშლის ყველა მსგავს ცვლადს.

**დაწერეთ ფუნქცია რომელიც არგუმენტად მიიღებს დატა ფრეიმს, მიღებულ დატაფრემიდან წაშლის ყველა ისეთ ცვლადს რომელის ცარიელი მნიშნველობებიც აღებატება 70% და დააბრუნებს გაწმენდილ დატაფრეიმს**

In [48]:
df = dropping_70_nans(df)

მონაცემების გაწმენდის შემდგომი ეტაპია, დუბლირებული ჩანაწერების წაშლა. შინაარსობრივადაც, რომ დავუკვირდეთ ძალიან უცნაური იქნება რეგრესიულ ამოცანაში ორი ერთმანეთის იდენტური სტრიქონი. ჩვენი მონაცემების მაგალითზე რომ ვთქვათ: ორი ტაქსი რომლებიც წარმოადგენდნენ ერთსა და იმავე კომპანიას. მგზავრები აიყვანეს იდენტურ დროს, დანიშნულების ადგილას მივიდნენ იდენტურ დროს, იდენტური მგზავრების რაოდენობით, ინდენტური გეოგრაფიულ ლოკაციებზე. რა თქმა უნდა ასეთი ჩანაწერი თითქმის დარწმუნებით შეგვიძლია ვთქვათ, რომ გაფუჭებულია და მოდელსაც აბნევს. შესაბამისად მსგავსი ტიპის ჩანაწერები უნდა წავშალოთ, როგორც რეგრესიულ ისე კლასიფიკაციის ამოცანებში. განსხვავება მხოლოდ ერთია: რეგრესიული ამოცანის დროს დუბლირებას ვუწოდებთ საპროგნოზო ცვლადის ჩათვლით, ხოლო კლასიფიკაციაში მის გარეშე. ლოგიკურიცაა, თუ ვპორგნოზირებთ ორ-კლასიან ცვლადს, რომ გამეორდეს უცნაურია არაფერია. ხოლო უწყვეტი ცვლადის ზუსტი განმეორება უკვე ეჭვქვეშ აყენებს მონაცემის სისწორეს.

ამ ოპერაციისთვის პანდას გააჩნია შესაბამისი ფუქნცია რომელიც თავიდან გვაცილებს დუბლირებულ ჩანაწერებს. იმისათვის, რომ დავინახოთ იყო თუ არა ჩვენს მონაცემებში მსგავსი ჩანაწერები, აღნიშნული ფუქნციის გამოყენებამდე და გამოყენების შემდგომ დავბეჭდოთ ცხრილის ჩანაწერების რაოდენობა.

In [49]:
df.shape[0]

1458644

In [50]:
df = df.drop_duplicates()

In [51]:
df.shape[0]

1458644

საბედნიეროდ ჩვენს ჩანაწერებში დუბლირებული მონაცემები არ აღმოჩნდა, თუმცა აღნიშნული ნაბიჯის შესრულება აუცილებელია. წინააღმდეგ შემთხვევასი შესაძლოა უარყოფითი გავლენა მოვახდინოთ მოდელის სიზუსტეზე.


წინა თავში როდესაც მონაცემებს ვიკვლევდით, განცალკევებულმა მონაცემება საკმაოდ შეგვიშალა ხელი. ზოგადად როგორც ანალიკურ ისე მოდელირების ნაწილში განცალკევებული მონაცემებთან გამკლავება უაღრესად მნიშნველოვანია. უკეთ რომ ავხსნათ, ნებისმიერი მოდელი უნდა იყოს განზოგადებული და ჩვენი მოლოდინიც უნდა იყოს რომ "უმეტეს" შემთხვევაში მუშაობდეს. შესაბამისად თუ განცალკევებულ (გამონაკლის) მონაცემებს არ შევიყვანთ მოდელში და მასაც არ ეცოდინება მსგავსი ტიპის ჩანაწერებზე კარგად პროგნოზირება, პრობლემა არ არის. მაგრამ პირიქით პრობლემა, რადგან როგორც აღვნიშნედ ისინი ხელს გვიშლიან როგორც ანალიზის ფაზაში დასკვნების გაკეთებაში, ისე "აბნევენ" ბევრ ალგორითს. ამიტომაც უმჯობესია ყოველთვის გამოვიკვლიოთ და თუ საჭიროა წავშალოთ კიდეც.

ერთი მარტივი და ზოგადი წესი შეგვიძლია დავნერგოთ, რომელსაც ხშირად მიმართავენ. დავიტოვოთ მხოლოდ ისეთი მონაცემები რომელიც ცვლადის 25-ე და 75- კვანტილებიდან 1.5-ჯერ კვარტილთაშორისი სიდიდით არიან დაშორებუელბი. ასეთ მეთოდს შესაძლოა ბევრი ისეთი ჩანაწერიც "ემსხვერპლოს" რომელიც დიდად არ იმსახურებდა წაშლას, თუმცა მეორეს მხრივ დაზღვეულები ვართ, რომ განცალკევებული მონაცემები ხელს არ შეგვიშლიან. ჩვენც დავწეროთ ფუნქცია რომელიც ცხრილის და სვეტის გადაცემებით, გაფილტრულ ცხრილს დააბრუნებს.

In [93]:
def drop_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

სანამ ცვლადს საკუთარ "გაფილტრულ" თავს გადავაწერთ, მანამდე შევამოწმოთ თუ რამდენ მონაცემს წაშლის ჩვენი ფუქნცია თითოეული ცვლადისთვის. თავდაპირველად შევქმნათ ასეთი ცვლადების ჩამონათვალი.

In [100]:
outliers_to_chekc = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
                     'dropoff_latitude', 'trip_duration']

In [101]:
for i in outliers_to_chekc:
    print(i, drop_outliers(df, i).shape[0] / df.shape[0])

pickup_longitude 0.942191514859006
pickup_latitude 0.9638410743128549
dropoff_longitude 0.8515827028390752
dropoff_latitude 0.8553375600900562
trip_duration 0.9490458261234407


როგორც ხედავთ დანიშნულების ადგილის გრძედსა და განედზე თითქმის 15% მოექცა ჩვენი წესით განსაზღვრუ განცალკევებულ მონაცემებად. თუმცა იქედან გამომდინარე, რომ ზოგადად გრძედი და განედი სპეციფიური მონაცემებია და მას სტარტული გზა ნაკლებად ერგება, სწორი იქნება თუ მთლიან ცხრილს მხოლოდ მგზავრობის ხანგრძლივობით გავფილტრავთ.

In [112]:
# დასაწრმუნებლად შევამოწმოთ რამდენად სწორად იმუშავა ფუნქციამ
# df_1 = drop_outliers(df, 'trip_duration')
# df_1.shape[0] / df.shape[0]

0.9490458261234407

ამ ეტაპზე შეგვიძლია ვთქვათ, რომ განცალკევებულ მონაცემებს გავუმკლავდით. თუმცა უაღრესად მნიშნველოვანია ამ პროცესში ცვლადებისა და ამოცანის შინაარსის გათვლისწინება და ვნახეთ რომ არ შეიძლება რაიმე ერთი ზოგადი წესი მოვარგოთ ყველაფერს.

ჩვენ რამდენიმე აბზაცის წინ წავლშალეთ დუბლირებული მონაცემები. თუმცა ეს დუბლირება მოიცავთა მთლიანი სტრიქონის ერთნაირობას. არაფერი გვითქვამს იმ შემთხვევაზე, როდესაც ცვლადი არის კონსტანტა. რა თქმა უნდა ასეთი ცვლადები არაფრის მომცემია და ისინიც უნდა წავშალოთ მონაცემების დამუშავების პროცესში. მართალია ჩვენს ამოცანაში ასეთი ცვლადი არ არის, თუმცა მაინც შემოგთავაზებთ ფუნქციას რომელიც მომავალში აღნიშნული პრობლემის გადაჭრაში დაგეხმარებათ. 

მსგავსი შემთხვევის შესამოწმებლად შეგვიძლია ვარიაცია გამოვიყენოთ. თუ ცვლადს აქვს ნულოვანი ვარიაცი ეს ავტომატურად ნიშნავს, რომ მისი ყველა ჩანაწერი არის ერთმანეთის ტოლი. შესაბამისად როგორც აღვნიშნეთ ასეთი ცვლადები არანაირი ინფორმაციის მატარებლები არ არიან მოდელისთვის. მაგალითად ჩვენ მონაცემებში რომ გვქონდეს ცვლადი "ქვეყანა", თითოეული მისი ჩანაწერი იქნებოდა აშშ, რაც მოდელს არანაირ ღირებულებას არ შესძენდა.

ზოგადად მანქანური დასწავლის ვერც ერთი ალგორითმი ვერ აღიქვამს ტექსტურ ინფორმაციას. შესაბამისად ასეთი ცვლადების არსებობის შემთხვევაში საჭიროა მათი კოდირება მანქანისთვის გასაგებ ენაზე. მაგალითად თუ გვაქვს ცვლადი სადაც წერია ძაღლი ან კატა, ეს უნდა გადავთარგმნოთ 1 ან 0, ან კიდევ რაიმე სხვა რიცხვით ფორმაში. ასევე შეუძლებელია ტექსტურ ცვლადზე ვარიაციის დათვლაც. სწორად ამიტომ ჯერ ჩვენ მოვახდენთ ცვლადების კოდირებას და შემდგომ შევამოწმებთ ნულოვან ვარიაციაზე.

In [132]:
le = preprocessing.LabelEncoder()
le.fit(df['store_and_fwd_flag'])
df['store_and_fwd_flag'] = le.transform(df['store_and_fwd_flag'])

In [140]:
def zero_variance_columns(df):
    var_thres = VarianceThreshold(threshold = 0)
    df1 = df._get_numeric_data()
    var_thres.fit(df1)
    constant_columns = [column for column in df1.columns
                    if column not in df1.columns[var_thres.get_support()]]
    df = df.drop(constant_columns, axis = 1)
    return constant_columns

In [141]:
zero_variance(df)

[]

მონაცემების წმენდის შემდეგი ეტაპია მაღალ-კორელირებული ცვლადების წაშლა. ამ პროცესს ორი ძირითადი მიზეზი აქვს: 1) მაპროგნოზირებელი ცვლადების მაღალი კორელაცია მიუთითებს იმაზე, რომ ისინი ერთი ინფორმაციის მატარებლები არიან. შესაბამისად გამოთვლითი რესურსის მეტად ეფექტურად გამოსაყენებლად ყოველთვის ჯობია მოვიცილოთ ყველა ისეთი ცვლადი, რაც არ არის ღირებული ინფორმაციის მატარებელი. 2) საპროგნოზო ცვლადთან მაღალი კორელაცია ნიშნავს, რომ ფაქტიურად ამხსნელი ცვლადი გვაქვს მონაცემებში. ეს შეიძლება ნიშნავდეს იმას, რომ იმდენად მარტივი დამოკიდებულება საპროგნოზო და მაპროგნოზირებელ ცვლადებს შორის, რომ საერთოდ არ არის მოდელი საჭირო. ან მაპროგნოზირებელ ცვლადებში გაპარული გვაქვს ისეთი "მომავლის" ინფომაცია რაც რეალურ ცხოვრებაში აღარ გვექნება (ჩვენი ამოცანის მაგალითზე ასეთი ცვლადი იქნებოდა მგზავრობის დასრულება), შესაბამისად ასეთი ცვლადები ყოველთვის უნდა ამოვაგდოთ მოდელირების პროცესიდან.

აღნიშნული ოპერაციის ჩასატარებლად ჩვენ შემდეგ წესს გთავაზობთ: მაპროგოზნირებელი ცვლადებისთვის გამოვიყენოთ 0.85 კორელაციის ზღვრად, ხოლო საპრონოზოსთან 0.9.

In [ ]:
# კორელაციური მატრიცის შექმნა აბსოლუტური მნიშვენლობებით
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df.drop(to_drop, axis=1, inplace=True)

In [120]:
def drop_high_corr (df, column_list, threshold):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    df = df.drop(to_drop, axis = 1)
    return df

იმისათვის რომ წარმოდგენა შევიქმნათ თუ რა ცვლადებს წაშლის ჩვენი შექმნილი ფუქნცია, ჯერ დავითვალოთ კორელაციული მატრიცა და თვალით ვნახოთ რა ხდება ჩვენს ცხრილში.

In [119]:
df.corr()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
passenger_count,1.000000,0.002154,-0.005254,-0.000557,-0.003323,0.008344
pickup_longitude,0.002154,1.000000,0.022568,0.796916,0.103825,0.026542
pickup_latitude,-0.005254,0.022568,1.000000,0.119777,0.499523,-0.029204
dropoff_longitude,-0.000557,0.796916,0.119777,1.000000,0.121336,0.016737
dropoff_latitude,-0.003323,0.103825,0.499523,0.121336,1.000000,-0.019741
trip_duration,0.008344,0.026542,-0.029204,0.016737,-0.019741,1.000000


როგორც ხედავთ ყველაზე მაღალი კორელაცია არის 0.79 რაც მგზავრობის აყვანისა და დასრულების გრძედებს შორისაა. რა თქმა უნდა ეს არაფრის მთქმელია, რადგა საქმე გეოლოკაციურ ცვლადებთან გვაქვს და ისევე როგორც განცალკევებული მონაცემების შემთხვევაში გამოვტოვეთ ასეთი ტიპი ცვლადები, კორელაციური დამოშავების შემთხვევაშიც იგივე უნდა ვქნათ. ჩვენ მიერ შემოთავაზებული კორელაციური ზღვრები, გავლენას არ მოახდენს. შესაბამისად ჩვენი ამოცანისთვის ეს ნაბიჯი შეგვიძლია გამოვტოვოთ.

მგზავრობის ხანგრძლივობის ხანგრძლიოვბის 2 საათზე მოჭრამ, არა მხოლოდ მკვეთრად გამოხატული უცნაურობები გააქრო, არამედ ზოგადად მთელი გრაფიკი უფრო დააგლუვა. 

ზოგადად მსგავსი ტიპის ოპერაციას განცალკევებული მონაცემების გადაყრას ვუწოდებთ ხოლმე. ის ერთ-ერთი უმნიშნველოვანესი ნაწილია, როდესაც გვსურს როგორც მონაცემების სწორად გაანალიზება ისე მოდელირება. თუმცა ამ პროცესის გარდა კიდევ ბევრი ნაბიჯია გასავლელი იქამდე სანამ მოდელის პირველ დამუშავებულ ვერსიას გავუშვებთ. ამ პროცესების ერთობლიობას მონაცემების მომზადება ეწოდება და იდეურად ის მონაცემების კვლევითი ანალიზის განუყოველი ნაწილია. თუმცა ჩვენ გვურს ეს შემდგომ თავში გავიაროთ, რათა უფრო მეტ დეტალებზე კონცენტრირება შევძლოთ და სიახლეებიც ნაბიჯ-ნაბიჯ ავითვისოთ.